In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import math
import cv2
import numpy as np

In [ ]:
import cv2
import os

# Specify the folder containing images (relative path)
folder_path = './vehicle-dataset/motorcycle/'  # Assuming the folder is named 'images' and is in the root of the project

# Create a list to store loaded images
images = []

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    # Construct the full file path
    img_path = os.path.join(folder_path, filename)

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Check if the image was successfully loaded
    if img is not None:
        images.append(img)
    else:
        print(f"Failed to load {img_path}")

print(f"Loaded {len(images)} images from {folder_path}")

In [ ]:
min_contour_area = 100
for i in range(len(images)):
    grayscale_image = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY) # Keep batch dimension

    # Apply binary threshold
    ret, thresh = cv2.threshold(grayscale_image, 110, 255, cv2.THRESH_BINARY)  # Scale to [0, 255]

    # Visualize the binary image
    #cv2.imshow('Binary image', thresh)
    #cv2.waitKey(0)  # Wait for a key press

    # Save the thresholded image
    cv2.imwrite(f'./motorcycle-binary/image_thres_{i}.jpg', thresh)
    contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contour_stats = []                                   
    # draw contours on the original image
    image_copy = images[i].copy()
    #print(f"Image shape for drawing contours: {image_copy.shape}, dtype: {image_copy.dtype}")  # Debugging line
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, closed=True)
        if area >= min_contour_area:
            contour_stats.append({
                "Contour ID": len(contour_stats) + 1,
                "Area": area,
                "Perimeter": perimeter
            })
        # Generate a random color for the current contour
            color = tuple(np.random.randint(0, 256, size=3).tolist())
            
            # Draw the current contour with the random color
            cv2.drawContours(image=image_copy, contours=[contour], contourIdx=-1, color=color, thickness=2, lineType=cv2.LINE_AA)
    # see the results
    #cv2.imshow('None approximation', image_copy)
    #cv2.waitKey(0)
    if i==117:
        print(f"Image {i + 1}:")
        print(f"Total number of contours detected: {len(contours)}")
        print(f"Contours with area >= {min_contour_area}: {len(contour_stats)}")
        for stat in contour_stats:
            print(f"Contour {stat['Contour ID']}: Area = {stat['Area']:.2f}, Perimeter = {stat['Perimeter']:.2f}")
        print("-" * 40)
    cv2.imwrite(f'./motorcycle-contour/contours_none_image_{i}.jpg', image_copy)